## Exploration of Gradient Descent and Neural Networks

#### Brendan Schlaman (July 2023)

**Technical Requirements:**
- Jupyter (with MathJax support for $\LaTeX$)
- `python>=3.9`
- `numpy`
- `prettytable`

**Human Requirements:**
- Reasonable understanding of linear algebra and vector calculus

---

### Introduction

The purpose of this notebook is to incrementally translate the mathematics of
gradient descent into code in the context of a simple neural network.

We will use only simple math libraries like `numpy` to gain a more fundamental intuition of the concepts; these concepts are
abstracted away in more purpose-built libraries like `pytorch` or TensorFlow.

We will start with the simplest possible neural network: a 1-wide ($m=1$), single layer ($L = 1$) network, and build up from there.

### Notation

The following conventions will be used:

| Variable or symbol | Definition |
|---|---|
| $\mathbf{W}^{(l)}$ | The weights matrix that describes inputs to layer $l$.  $\mathbf{W}^{(l)}$ is an $(m \times n)$ matrix, where $m$ is the size of layer $l$, and $n$ is the size of layer $l-1$. |
| $\mathbf{w}$ | Same as above, but when $\mathbf{w}$ is a vector (single column matrix).  This is the case where there is a single neuron for a particular layer. |
| `W1` | When using a fixed number of layers *L*, `W1` will be the weights feeding into the first hidden layer (or the output layer).  `Wl` in the code corresponds to $w^{(l)}$ in the math notation. |
| $\mathbf{b}$ | A single bias matrix that contributes to a hidden layer.  $\mathbf{b}$ is a $(m \times 1)$ matrix, where $m$ is the size (width) of the associated hidden layer.  |
| `b1` | When using a fixed number of layers *L*, `b1` will be the weights feeding into the first hidden layer (or the output layer).  `bn` in the code corresponds to $b^{(n)}$ in the math notation. |
| `z` | A preactivated layer.  It is the result of the linear combination (weights times inputs plus bias) before an activation function is applied. |
| `a` | An activated layer, i.e. $\sigma (z)$.  The activated layer becomes the input to the next layer. |
| $\mathbf{X}$ | The training examples matrix.  Each row represents an example, and each column represents a feature. |
| $\mathbf{x}$ | A single input vector to a layer $l$; $z^{(l)} = f^{(l)}(\mathbf{x})$.  |
| $\mathbf{Y}$ | The training targets.  $\mathbf{Y_i}$ is the target of example $\mathbf{X_i}$.  The columns represent the target probability distribution over the categories of the data.  In our case, this will be a *one-hot* encoding of the expected label. |
| $L$ | The depth of the neural network; the number of hidden layers plus the output layer. |
| $C_i$ | The cost of the neural network for training example $i$. |
| $j$ | Index of the current layer, the layer denoted in the superscript. |
| $k$ | Index of the previous layer, the layer before the layer denoted in the superscript. |

<br>

> Note that the *input layer* is treated as the “0th” layer and does not have wieghts or biases.
> Imagine each layer $l$ "owning" the weights $\mathbf{W}^{(l)}$ and biases $\mathbf{b}^{(l)}$
> that contribute to its activation.

### A note on NN architecture and dimensionality

One of the things that confused me the most as I first got into neural networks is
how the 4 "dimensionalities" of neural network training (3 *architecture dimensions* x num examples)
should be represented in your datastructures.  Luckily, math should work out the same for whatever
configuration you choose, as long as that configuration is consistent, but I believe some representations
are better (and indeed perhaps more performant) than others.  I landed on the following
representation after some deliberation and consultation with online sources, and
I think it does the best job of preserving intuitions while following convention.
Just be aware that other materials, tutorials, or libraries may structure their data differently.

> Note that these dimensionalities are separate from the **hyperparameters** of the system,
> like the learn rate and choice of activation function(s).  For our exmples, the
> representations for these parameters will be clear in the code.

| Parameter Type | Dimension | Our notation | Datastructure representation |
|---|---|---|---|
| Input | Number of training examples | $N$ | Num rows in $\mathbf{X}$ and $\mathbf{Y}$ |
| Architecture | Depth of the NN | $L$ | Object of iteration.  We will iterate over each layer during a single forward or back propagagation cycle (or unroll and compute sequentially if $L$ is sufficiently small, using unique variables `Wl`, `bl`, `zl`, `al`, etc, where `l` $\in \{1,\dots,L\}$). |
| Architecture | Size of layer $l$ | $m^{(l)}$ | Num rows of $\mathbf{W}^{(l)}$ |
| Architecture (multiple values) | Size of layer $l-1$ | $m^{(l-1)}$ | Num columns of $\mathbf{W}^{(l)}$ |
| Input + Architecture¹ | Number of features; size of input layer | $m^{(0)}$ | Num columns of $\mathbf{X}$ |
| Input + Architecture¹ | Number of categories; size of output layer | $m^{(L)}$ | Num columns of $\mathbf{Y}$; size of $\mathbf{b}^{(L)}$; num rows of $\mathbf{W}^{(L)}$ |

<br>

> ¹These parameters can be inferred from the 3 *architecture* parameters above them,
> i.e. they are a restating of the previous parameters.
> I've included them here for illustration purposes.

For example, one could easily imagine iterating over examples $\mathbf{x_i}$
and keeping track of a map from layer $l$ to parameter
$\mathbf{W}^{(l)}, \mathbf{b}^{(l)}, \mathbf{z}^{(l)}, \mathbf{a}^{(l)}$.

### Key equations

#### Neural network definitions

$$
    \mathbf{z}^{(L)} = \mathbf{W}^{(L)}\mathbf{a}^{(L-1)} + \mathbf{b}^{(L)}
$$
$$
    \mathbf{a}^{(L)} = \sigma (\mathbf{z}^{(L)})
$$

> Going forward, I'll drop the boldface notation for vectors and matrices.  $W$ is always a matrix, and $b$, $z$, $w$ are always vectors.


#### Computing loss

We'll use the common **Mean Squared Error (MSE)** loss function:

$$
    C_i = \frac{1}{N}\sum_{j=0}^{N-1}(y_j - a_j^{(L)})^2
$$

where $C_i$ is the loss from a single example, $x_i$.
This is precicely the function we will try to minimize with gradient descent.
Note that $\frac{1}{N}$ simply scales the loss and does not change which inputs minimize the function.

We are interested in how the cost of the network changes with respect to it's inputs, $W$ and $b$.
The chain rule gives us our answer:

$$
    \frac{\partial C_i}{\partial W_{jk}^{(l)}} =
        \frac{\partial C_i}{\partial a_j^{(l)}}
        \frac{\partial a_j^{(l)}}{\partial z_j^{(l)}}
        \frac{\partial z_j^{(l)}}{\partial W_{jk}^{(l)}}
$$
$$
    \frac{\partial C_i}{\partial b_j^{(l)}} =
        \frac{\partial C_i}{\partial a_j^{(l)}}
        \frac{\partial a_j^{(l)}}{\partial z_j^{(l)}}
        \frac{\partial z_j^{(l)}}{\partial b_j^{(l)}}
$$

It's also helpful to consider the derivative of $C_i$ with respect to $a_j^{(l)}$,
since through that derivative, we get access future layers' ($l + 1, 2, \dots$) weights and biases.
Notice that the cost function is influenced in $n_{l+1}$ different ways for each $a_j^{(l)}$,
so we use the *multivariable chain rule* to account for all the paths between $a_j^{(l)}$ and the next layer.
Remember that when context switches from $l$ to $l + 1$, the indices $j, k$ are redefined in relation to that new context.

$$
    \frac{\partial C_i}{\partial a_j^{(l)}} =
        \sum_{j=0}^{n_{l+1}-1}
        \frac{\partial C_i}{\partial a_j^{(l+1)}}
        \frac{\partial a_j^{(l+1)}}{\partial z_j^{(l+1)}}
        \frac{\partial z_j^{(l+1)}}{\partial a_j^{(l)}}
$$

The total loss derivative with respect to $W^{(L)}$ is the average of the losses $C_i$ from all examples $x_i$.

$$
    \frac{\partial C}{\partial W^{(L)}} = \frac{1}{n} \sum_{i=0}^{n-1} \frac{\partial C_i}{\partial W^{(L)}}
$$

The full gradient of the loss function over the entire network $\nabla C$ comprises the above derivatives for all layers $l$.

$$
\nabla C =
    \begin{bmatrix}
    \frac{\partial C}{\partial W^{(1)}} \\[1.5ex]
    \frac{\partial C}{\partial b^{(1)}} \\[1.5ex]
    \vdots \\[1.5ex]
    \frac{\partial C}{\partial W^{(L)}} \\[1.5ex]
    \frac{\partial C}{\partial b^{(L)}}
    \end{bmatrix}
$$


Cross entropy loss:

$C = -\sum_{j=1}^{K} y_j \log(\hat{y}_j)$

$C = -\frac{1}{N} \sum_{i=1}^{N} \sum_{j=1}^{K} y_{ij} \log(\hat{y}_{ij})$

#### Nonlinear activation functions

The standard softmax function $\sigma : \mathbb{R}^K \mapsto (0, 1)^K; \; K \geq 1$:

$$
    \sigma(\mathbf{z})_i =
        \frac{e^{z_i}}{\sum_{j=1}^{K} e^{z_j}}
        \text{ for }
        1, \dots, K
        \text{ and }
        \mathbf{z} = (z_1, \dots, z_K) \in \mathbb{R}^K
$$

> The symbols used above are consistent with most online materials,
> where $i$ represents an index of the elements of $\mathbf{z}$,
> not an example index.

For this neural network, we'll be using **ReLU** as our activation function.

### Simpler cases

It's helpful to look at some of the math in edge-case neural networks.

In the case of a single output, our preactivation function can be written as

$$
    z = f(\mathbf{a}) = \mathbf{w} \cdot \mathbf{a} + b
$$

For a single layer ($L = 1$), 1-wide ($m = 1$) network, the partial derivatives of loss are:

\begin{align*}
    \frac{\partial C_i}{\partial w^{(L)}} =&&
        \frac{\partial C_i}{\partial a^{(L)}}
        \frac{\partial a^{(L)}}{\partial z^{(L)}}
        \frac{\partial z^{(L)}}{\partial w^{(L)}}
        &= 2(a^{(L)} - y) \sigma'(z^{(L)}) a^{(L-1)} \\
    \frac{\partial C_i}{\partial b^{(L)}} =&&
        \frac{\partial C_i}{\partial a^{(L)}}
        \frac{\partial a^{(L)}}{\partial z^{(L)}}
        \frac{\partial z^{(L)}}{\partial b^{(L)}}
        &= 2(a^{(L)} - y) \sigma'(z^{(L)}) \\
    \frac{\partial C_i}{\partial a^{(L-1)}} =&&
        \frac{\partial C_i}{\partial a^{(L)}}
        \frac{\partial a^{(L)}}{\partial z^{(L)}}
        \frac{\partial z^{(L)}}{\partial a^{(L-1)}}
        &= 2(a^{(L)} - y) \sigma'(z^{(L)}) w^{(L)}
\end{align*}


### 1-Wide, single layer Neural Network ($L = 1$; $m^{(L)} = 1$)

Start with the simplest NN - an input layer connected to a single output node (one category).

In this simple case, the output layer is simply

$$
    a(\mathbf{x}) = σ(\mathbf{w} \cdot \mathbf{x} + b)
$$

We can take advantage of vector calculus to find our partial derivatives, using the intermediate variable $z = \mathbf{w} \cdot \mathbf{x} + b$.

$$
    \frac{\partial a^{(l)}}{\partial \textbf{w}} =
    \frac{\partial a^{(l)}}{\partial z}
    \frac{\partial z}{\partial \textbf{w}}
$$

Looking at $\frac{\partial z}{\partial \textbf{w}}$,

\begin{align*}
    \frac{\partial z}{\partial \textbf{w}}
    &= \frac{\partial}{\partial \textbf{w}} \left(\text{sum} (\mathbf{w} \otimes \mathbf{x}) + b\right) \\
    &= \frac{\partial \ \text{sum} (\mathbf{v})}{\partial \textbf{v}} \frac{\partial \mathbf{v}}{\partial \textbf{w}}
    + \frac{\partial}{\partial \textbf{w}} b \ ; \ \mathbf{v} = \mathbf{w} \otimes \mathbf{x} \\
    &= \vec{1}^T \text{diag}(\mathbf{x}) + \vec{0}^T \\
    &= \mathbf{x}^T
\end{align*}

Finally, we have our answer:

\begin{align*}
    \frac{\partial a^{(l)}}{\partial \textbf{w}} &= \frac{\partial a^{(l)}}{\partial z} \mathbf{x}^T \\
    \frac{\partial a^{(l)}}{\partial b} &= \frac{\partial a^{(l)}}{\partial z}
\end{align*}

$$
    \frac{\partial C}{\partial b}
    = \frac{\partial C}{\partial a} \frac{\partial a}{\partial z}
    = - \frac{\partial a}{\partial z} \frac{2}{N}\sum_i y_i - \sigma(\mathbf{w} \cdot \mathbf{x_i} + b)
    = \frac{\partial a}{\partial z} \frac{2}{N}\sum_i \sigma(\mathbf{w} \cdot \mathbf{x_i} + b) - y_i
$$

$$
    \frac{\partial C}{\partial \textbf{w}}
    = \frac{\partial C}{\partial a} \frac{\partial a}{\partial z} \mathbf{x}^T
    = - \frac{\partial a}{\partial z} \frac{2}{N}\sum_i (y_i - \sigma(\mathbf{w} \cdot \mathbf{x_i} + b)) \mathbf{x_i}^T
    = \frac{\partial a}{\partial z} \frac{2}{N}\sum_i (\sigma(\mathbf{w} \cdot \mathbf{x_i} + b) - y_i) \mathbf{x_i}^T 
$$


With a learning rate of $\alpha$, our gradient descent algorithm will look like:

$$
    \mathbf{w}_{t+1} = \mathbf{w}_t - \alpha \frac{\partial C}{\partial \textbf{w}} \\[1.5ex]
    b_{t+1} = b_t - \alpha \frac{\partial C}{\partial b}
$$




#### Setup

First we'll define some utility functions that help us build the data for our example.
The nature of these functions is chosen arbitrarily.

In [ ]:
import random


def init_single_category_data(d: int):
    """Generates random training data with dimensionality `d`.
    The label is also chosen arbitrarily.
    """
    data_size = 10
    labels = [9] * data_size  # randomly chose a desired label
    examples = [[random.randrange(-9, 10) for _ in range(d)] for _ in range(data_size)]
    return list(zip(examples, labels))


In [ ]:
import numpy as np

input_dimensionality = 3

examples, labels = zip(*init_single_category_data(input_dimensionality))
X, Y = np.array(examples), np.array(labels)

m1 = 1 # the width of our only layer! (remember, we don't count the input layer)

def init_params():
    w1 = np.random.randn(input_dimensionality) * 0.01
    b1 = np.random.randn(m1) * 0.01
    return w1, b1

w1, b1 = init_params()

X, Y, w1, b1

We'll use the same `ReLU` function throughout, regardless of layer width.

In [ ]:
def ReLU(Z: np.ndarray) -> np.ndarray:
    return np.maximum(0, Z)

def deriv_ReLU(Z: np.ndarray) -> np.ndarray:
    return Z > 0

Forward propagation is simple.
Matrix multiply `w1` and `X.T` and add `b1`.

In [ ]:
def forward_prop(
    X: np.ndarray,
    w1: np.ndarray,
    b1: np.ndarray,
):
    Z1 = w1.dot(X.T) + b1
    A1 = ReLU(Z1) # 1D ReLU
    # print(w1, x, w1.dot(x), z1, a1)
    return Z1, A1

def mean_squared_error_loss(Y: np.ndarray, A: np.ndarray) -> float:
    return np.sum(np.square(A-Y)) / len(A)


Next, define a backpropagation method

In [ ]:
def gradient(X: np.ndarray, Y: np.ndarray, Z: np.ndarray, A: np.ndarray):
    n = len(Y)
    dZ = 2*(A-Y) * deriv_ReLU(Z)
    # TODO (2023.07.03): do I need this 1/n?
    dW = 1/n * dZ.T.dot(X)
    db = 1/n * np.sum(dZ)
    return dW, db

def descent(W1, b1, dW, db):
    alpha = 0.01
    W1 = W1 - alpha * dW
    b1 = b1 - alpha * db
    return W1, b1

In [ ]:
from prettytable import PrettyTable

def pretty_print_parameters(**kwargs):
    def yel(s: str) -> str:
        return f"\033[93m{s}\033[0m"

    pt = PrettyTable()
    pt.field_names = ["Parameter", "Value"]
    pt.max_width["Value"] = 80
    pt.align = "l"

    for k, v in kwargs.items():
        pt.add_row([yel(k), v])

    print(pt)

def run(X: np.ndarray, Y: np.ndarray):
    W1, b1 = init_params()
    for i in range(14000):
        Z1, A1 = forward_prop(X, W1, b1)
        loss = mean_squared_error_loss(Y, A1)
        dW, db = gradient(X, Y, Z1, A1)
        W1, b1 = descent(W1, b1, dW, db)
        if i % 500 == 0:
            print(f"===== {i} ======")
            pretty_print_parameters(W1=W1, b1=b1, A1=A1, loss=loss, dW=dW, db=db)


Try running the model.

In [ ]:
run(X, Y)

Talk about overfitting!  The network has found a trivial solution to the problem: $\mathbf{w} = \vec{0} ; \ b = 9$.

Great to know it works!  Now let's try a slightly harder problem for the network.

In [ ]:
Y = np.array([3, 4, 1, -8, -7, 4, 4, -8, 3, 1])
run(X, Y)

Well, it certainly converges; on what, I'm not sure.  Who knows if the minima is local or global; it's hard to tell when the training data $X$ was generated randomly.

Time to step it up to the big leagues.  We will now increase two dimensions of our network: $L$ and $m^{(l > 0)}$.
This means we can build a real, functional neural network!  We'll be using the classic **mnist dataset** for our final project.

The network will be an $L = 3$ network (2 hidden layers, 1 output layer).  Layer sizes will be as follows:

| Layer $l$ | Size |
|---|---|
| $m^{(1)}$ | 16 |
| $m^{(2)}$ | 12 |
| $m^{(L)}$ | 10 (defined by number of categories) |

Luckily, the code won't change too much.  The only differences are:

1. Introduce parameter lists to hold weights, biases, preactivated neurons, and activated neurons for all layers.
1. $\mathbf{Y}$ becomes an $m \times n$ matrix, where each column is a one-hot encoding of the value of the label for the $i$th example.
1. $\mathbf{b}^{(l)}$ becomes a vector instead of a number.
1. $\mathbf{z}^{(l)}$ and $\mathbf{a}^{(l)}$ become matrices (`Z`, `A`) whose rows correspond to the training examples.
1. For our final activation function, we'll use **softmax** instead of ReLU.
   Softmax scales its input vector to a probability distribution while amplifying
   differences in values.  Ideally, our model outputs a 1 for the correct label
   and a 0 everywhere else.
1. We'll switch to using **cross-entropy loss** for our loss function since it plays so nicely with **softmax**.


Since we've introduced softmax, we'll need its derivative to perform gradient descent.

Softmax is defined as follows:

$$
    \text{softmax}(\mathbf{z})_i = \mathbf{\sigma}_i(\mathbf{z}) = \frac{e^{z_i}}{\sum_j e^{z_j}}
$$

We'll find its Jacobian, leveraging the fact that outputs of softmax are strictly positive.
(See Appendix for a wonderful intuition I discovered that helped me grasp the motivation behind Jacobian notation conventions.)

$$
    \frac{\partial}{\partial \mathbf{z}} \log (\boldsymbol{\sigma})
    = \frac{1}{\mathbf{\sigma}} \frac{\partial}{\partial \boldsymbol{\sigma}} \\[1.5ex]
    \frac{\partial}{\partial \boldsymbol{\sigma}}
    = \boldsymbol{\sigma} \frac{\partial}{\partial \mathbf{z}} \log(\boldsymbol{\sigma})
$$

Focusing in on a single $\sigma_i$ and $z_j$, we'll sum over $k$ to avoid polluting our namespace.

$$
    \log(\sigma_i) = z_j - \log\left(\sum_k e^{z_k}\right)
$$

We can write

$$
    \frac{\partial}{\partial z_j} \log(\sigma_i)
    = 1\{i = j\}-\frac{1}{\sum_k e^{z_k}} \frac{\partial}{\partial z_j} \sum_k e^{z_k}
    = 1\{i=j\}-\sigma_j
$$

So finally,


$$
    \frac{\partial \sigma_i}{\partial z_j}
    = \sigma_i \frac{\partial}{\partial z_j} \log(\sigma_i)
    = \sigma_i (1\{i=j\}-\sigma_j)
$$

$$
    J_{\text{softmax}} =

    \begin{bmatrix}
    \sigma_1(1-\sigma_1) & -\sigma_1\sigma_2 & \dots & -\sigma_1\sigma_n \\
    -\sigma_2\sigma_1 & \sigma_2(1-\sigma_2) & \dots & -\sigma_2\sigma_n \\
    \vdots & \vdots & \ddots & \vdots \\
    -\sigma_m\sigma_1 & -\sigma_m\sigma_2 & \dots & \sigma_m(1-\sigma_m)
    \end{bmatrix}
$$

Cross-entropy loss is defined as

$$
    L = -\sum\limits_{i}y_i \log(\sigma_i)
$$

And now, for the magic!

$$
    \frac{\partial L}{\partial z_j}
    = -\sum\limits_{i}y_i \frac{\partial}{\partial z_j} \log(\sigma_i)
    = -\sum\limits_{i}y_i 1\{i=j\}-\sigma_j \\
    = \sum\limits_{i}y_i\sigma_j - \sum\limits_{i}y_i 1\{i=j\}
    \frac{\partial L}{\partial z_j}
    = \left(\sum\limits_{i}y_i\sigma_j\right) - y_j
    = \sigma_j \left(\sum\limits_{i}y_i\right) - y_j
    = \sigma_j - y_j
$$

The last step is possible since the one-hot encoded $\mathbf{Y}_i$ sums to 1.

The entire expression simplifies beautifully:

$$
    \frac{\partial L}{\partial \mathbf{z}} = \boldsymbol{\sigma} - \mathbf{Y}
$$

Now we're ready for the code!

First, the `softmax` and `one_hot` functions described above.

In [154]:
def softmax(A: np.ndarray):
    return np.exp(A) / np.sum(np.exp(A))

In [155]:
def one_hot(Y: np.ndarray) -> np.ndarray:
    """Constructs a matrix whose rows represent
    the target probability distribution of categories
    for each row in Y, which in this case will be a
    1 in the column indexed by the desired label,
    and zeros everywhere else.
    """
    one_hot_Y = np.zeros((Y.size, np.max(Y) + 1))
    one_hot_Y[np.arange(Y.size), Y] = 1
    return one_hot_Y

Next, we'll define our forward prop and loss functions.
Note that the loss function is defined purely for debugging;
we'll directly use its derivative during gradient descent.

In [156]:
weights_l: list[np.ndarray] = []
biases_l: list[np.ndarray] = []
preactivated_layers: list[np.ndarray] = []
activated_layers: list[np.ndarray] = []

In [180]:
def forward_prop(
    X: np.ndarray,
    W_l: list[np.ndarray],
    b_l: list[np.ndarray],
):
    # these will store the preactivated and activated
    # neurons for each layer l
    Z_l, A_l = [], []

    # 1-index to match layer definitions
    for l, (W, b) in enumerate(zip(W_l, b_l), 1):
        Z = W.dot(X.T if l == 1 else A_l[l-2]) + b
        A = ReLU(Z) if l < len(W_l) else softmax(Z)
        Z_l.append(Z)
        A_l.append(A)
    return Z_l, A_l


def categorical_cross_entropy_loss(ohY: np.ndarray, A: np.ndarray) -> float:
    assert np.all(
        (ohY.sum(axis=1) == 1) & ((ohY == 0) | (ohY == 1))
    )  # one-hot encoding
    return -np.sum(ohY * np.log(A), dtype=float)


Unlike in the $L = 1$ example, we actually have a back propagation now.

In [178]:
def gradient_back_prop(
    X: np.ndarray,
    ohY: np.ndarray,
    W_l: list[np.ndarray],
    Z_l: list[np.ndarray],
    A_l: list[np.ndarray],
):
    n = len(Y)
    dW_l = []
    db_l = []
    prev_dZ = np.zeros(0)
    for l, (Z, A) in enumerate(zip(reversed(Z_l), reversed(A_l))):
        l = len(A_l) - l
        dZ = W_l[l + 1].dot(prev_dZ) * deriv_ReLU(Z) if l < len(A_l) else (A - Y)
        dW_l.append(dZ.T.dot(A_l[l - 2].T if l > 0 else X.T))
        db_l.append(np.sum(dZ))
        prev_dZ = dZ
    return dW_l[::-1], db_l[::-1]


def descent(W_l, b_l, dW_l, db_l):
    alpha = 0.01
    W_l = W_l - alpha * dW_l
    b_l = b_l - alpha * db_l
    return W_l, b_l


In [168]:
num_features = len(X.T)
num_categories = len(Y)

NN_ARCHITECTURE = {
    # layer: size
    1: 16,
    2: 12,
    3: num_categories,
}

def init_params():
    def gen_rand_array(m: int, n: int):
        return np.random.randn(m, n)

    W_l, b_l = [],[]

    for l, size in NN_ARCHITECTURE.items():
        W_l.append(gen_rand_array(size, NN_ARCHITECTURE[l-1] if l > 1 else num_features))
        b_l.append(gen_rand_array(size, 1))

    return W_l, b_l

def run(X: np.ndarray, Y: np.ndarray):
    W_l, b_l = init_params()
    print(W_l[0].shape)
    ohY = one_hot(Y)
    for i in range(1):
        Z_l, A_l = forward_prop(X, W_l, b_l)
        print(Z_l, A_l)
        return
        loss = mean_squared_error_loss(Y, Aoh1)
        dW, db = gradient(X, Y, Z1, A1)
        W1, b1 = descent(W1, b1, dW, db)
        if i % 500 == 0:
            print(f"===== {i} ======")
            pretty_print_parameters(W1=W1, b1=b1, A1=A1, loss=loss, dW=dW, db=db)



I will leave it to the reader to get the data that you want.

In [169]:
examples = np.array(x_train).reshape(len(x_train), -1)
labels = np.array(y_train)
input_layer_size = examples.shape[1]
hidden_layer_size = 10
output_layer_size = len(set(labels)) # number of categories

print(f"{examples.shape=}")
print(f"{labels.shape=}")
print(f"{input_layer_size=}")
print(f"{hidden_layer_size=}")
print(f"{output_layer_size=}")

DEV_SIZE = 11
X_dev = examples[:DEV_SIZE]
Y_dev = labels[:DEV_SIZE]

X_train = examples[DEV_SIZE:]
Y_train = labels[DEV_SIZE:]

X_dev.shape, Y_dev.shape, X_train.shape, Y_train.shape

examples.shape=(60000, 784)
labels.shape=(60000,)
input_layer_size=784
hidden_layer_size=10
output_layer_size=10


((11, 784), (11,), (59989, 784), (59989,))

In [181]:
X, Y = X_dev, Y_dev
print(Y)
run(X,Y)

[5 0 4 1 9 2 1 3 1 4 3]
(16, 784)
[array([[ 2.11095965e+02,  4.77321564e+02,  1.38502589e+03,
        -1.87762939e+02,  6.94156772e+02,  4.04232085e+02,
         1.22503969e+03,  4.61695285e+02,  5.04323765e+02,
         3.75479724e+03,  1.27700802e+03],
       [ 5.42894900e+03,  7.98203405e+02,  3.00564014e+00,
         2.89338578e+03,  9.52253518e+02,  8.18859196e+02,
         2.43356542e+03,  2.77203581e+03,  1.65310469e+03,
         3.50499746e+03,  3.40446136e+03],
       [ 3.64461631e+03,  5.01616971e+03,  3.75594328e+03,
         3.01469519e+03,  3.08432767e+03,  5.48618648e+03,
         8.28764971e+02,  6.42933068e+03, -2.80076333e+02,
         3.50314984e+03,  4.72071774e+03],
       [-5.66608076e+03, -3.82016283e+03, -1.51236201e+03,
        -1.77626982e+03,  1.61480669e+03, -3.82327937e+02,
        -2.07471468e+03, -1.58033519e+03, -9.16341535e+02,
        -1.30807244e+03, -2.74864360e+03],
       [-3.24248151e+02, -9.37275594e+02, -7.87282860e+02,
        -9.69516197e+02,  

/tmp/ipykernel_31982/387492863.py:2: RuntimeWarning: overflow encountered in exp
  return np.exp(A) / np.sum(np.exp(A))
/tmp/ipykernel_31982/387492863.py:2: RuntimeWarning: invalid value encountered in divide
  return np.exp(A) / np.sum(np.exp(A))


In [153]:
a = np.array([1,2,5,2,2])
a

array([1, 2, 5, 2, 2])

In [ ]:
softmax(a)

In [ ]:
sum(softmax(a))

In [135]:

prev_dZ = np.zeros(0)
prev_dZ

array([], dtype=float64)

## Appendix

### Further reading and materials that helped me make this notebook

- 3b1b series
- tds - part 1 - 4
- [Derivative of the Softmax Function and the Categorical Cross-Entropy Loss | by Thomas Kurbiel | Towards Data Science](https://towardsdatascience.com/derivative-of-the-softmax-function-and-the-categorical-cross-entropy-loss-ffceefc081d1)

#### A beautiful intuition behind the Jacobian

I had a beautiful "eureka" moment today.  I understand the motivation behind the **numerator layout** of the Jacobian matrix.
As a reminder, the Jacobian is defined as follows:

$$
    \mathbf{J} \mathbf{f}(\mathbf{x}) = \nabla \mathbf{f}(\mathbf{x}) =
    \begin{bmatrix}
    \nabla f_1(\mathbf{x}) \\[1.5ex]
    \nabla f_2(\mathbf{x}) \\[1.5ex]
    \nabla f_3(\mathbf{x}) \\[1.5ex]
    \vdots \\[1.5ex]
    \nabla f_i(\mathbf{x})
    \end{bmatrix}
    =
    \begin{bmatrix}
    \frac{\partial}{\partial \mathbf{x}} f_1(\mathbf{x}) \\[1.5ex]
    \frac{\partial}{\partial \mathbf{x}} f_2(\mathbf{x}) \\[1.5ex]
    \frac{\partial}{\partial \mathbf{x}} f_3(\mathbf{x}) \\[1.5ex]
    \vdots \\[1.5ex]
    \frac{\partial}{\partial \mathbf{x}} f_i(\mathbf{x})
    \end{bmatrix}
    = 
    \begin{bmatrix}
    \frac{\partial}{\partial x_1} f_1(\mathbf{x}) \frac{\partial}{\partial x_2} f_1(\mathbf{x}) \dots \frac{\partial}{\partial x_j} f_1(\mathbf{x}) \\[1.5ex]
    \frac{\partial}{\partial x_1} f_2(\mathbf{x}) \frac{\partial}{\partial x_2} f_2(\mathbf{x}) \dots \frac{\partial}{\partial x_j} f_2(\mathbf{x}) \\[1.5ex]
    \frac{\partial}{\partial x_1} f_3(\mathbf{x}) \frac{\partial}{\partial x_2} f_3(\mathbf{x}) \dots \frac{\partial}{\partial x_j} f_3(\mathbf{x}) \\[1.5ex]
    \vdots \\[1.5ex]
    \frac{\partial}{\partial x_1} f_i(\mathbf{x}) \frac{\partial}{\partial x_2} f_i(\mathbf{x}) \dots \frac{\partial}{\partial x_j} f_i(\mathbf{x})
    \end{bmatrix}
$$

The motivation for this configuration is clear when you consider the analog in linear transformations.
Consider an $(m \times n)$ matrix that defines a linear transformation from $\mathbb{R}^n \to \mathbb{R}^m$.
The columns are very interesting: the columns describe what happens to each unit vector in $\mathbb{R}^n$ under the transformation.
In other words, the element $x_{i,j}$ can be thought of as describing
*what contribution to the $i$ component of output vectors does the $j$ component of input vectors make?*

This aligns beautifully with the Jacobian!  $\frac{\partial}{\partial x_j} f_i(\mathbf{x})$ indeed describes
*what contribution a small change in the $j$ component of an input vector makes to the $i$ component of the vector function $\mathbf{f}$*.

Isn't that satisfying!?

Here's my first pass at working out the Jacobian of softmax.
It ends up being the same, but the logarithmic derivative is
much more clever.

$$
    J \boldsymbol{\sigma} = \frac{1}{\left(\sum_j e^{z_j}\right)^2}

    \begin{bmatrix}
    \sum\limits_{j \neq 1} e^{z_j} & -e^{z_1 + z_2} & \dots & -e^{z_1 + z_n} \\
    -e^{z_2 + z_1} & \sum\limits_{j \neq 2} e^{z_j} & \dots & -e^{z_2 + z_n} \\
    \vdots & \vdots & \ddots & \vdots \\
    -e^{z_m + z_1} & -e^{z_m + z_2} & \dots & \sum\limits_{j \neq m} e^{z_j}
    \end{bmatrix}
$$

If we define $\beta = \sum_j e^{z_j}$, we have

$$
    J \boldsymbol{\sigma} = - \frac{1}{\beta^2}

    \begin{bmatrix}
    e^{z_1} & e^{z_1 + z_2} & \dots & e^{z_1 + z_n} \\
    e^{z_2 + z_1} & e^{z_2} & \dots & e^{z_2 + z_n} \\
    \vdots & \vdots & \ddots & \vdots \\
    e^{z_m + z_1} & e^{z_m + z_2} & \dots & e^{z_m}
    \end{bmatrix}
    + \beta \mathbf{I}
$$

### mnist stuff

In [139]:

import numpy as np
import struct
from array import array
import os


class MnistDataloader(object):
    def __init__(
        self,
        training_images_filepath,
        training_labels_filepath,
        test_images_filepath,
        test_labels_filepath,
    ):
        self.training_images_filepath = training_images_filepath
        self.training_labels_filepath = training_labels_filepath
        self.test_images_filepath = test_images_filepath
        self.test_labels_filepath = test_labels_filepath

    def read_images_labels(self, images_filepath, labels_filepath):
        labels = []
        with open(labels_filepath, "rb") as file:
            magic, size = struct.unpack(">II", file.read(8))
            if magic != 2049:
                raise ValueError(
                    "Magic number mismatch, expected 2049, got {}".format(magic)
                )
            labels = array("B", file.read())

        with open(images_filepath, "rb") as file:
            magic, size, rows, cols = struct.unpack(">IIII", file.read(16))
            if magic != 2051:
                raise ValueError(
                    "Magic number mismatch, expected 2051, got {}".format(magic)
                )
            image_data = array("B", file.read())
        images = []
        for i in range(size):
            images.append([0] * rows * cols)
        for i in range(size):
            img = np.array(image_data[i * rows * cols : (i + 1) * rows * cols])
            img = img.reshape(28, 28)
            images[i][:] = img

        return images, labels

    def load_data(self):
        x_train, y_train = self.read_images_labels(
            self.training_images_filepath, self.training_labels_filepath
        )
        x_test, y_test = self.read_images_labels(
            self.test_images_filepath, self.test_labels_filepath
        )
        return (x_train, y_train), (x_test, y_test)


data_base_path = "../data/mnist"
training_images_filepath = os.path.join(
    data_base_path, "train-images-idx3-ubyte/train-images-idx3-ubyte"
)
training_labels_filepath = os.path.join(
    data_base_path, "train-labels-idx1-ubyte/train-labels-idx1-ubyte"
)
test_images_filepath = os.path.join(
    data_base_path, "t10k-images-idx3-ubyte/t10k-images-idx3-ubyte"
)
test_labels_filepath = os.path.join(
    data_base_path, "t10k-labels-idx1-ubyte/t10k-labels-idx1-ubyte"
)

mnist_dataloader = MnistDataloader(
    training_images_filepath,
    training_labels_filepath,
    test_images_filepath,
    test_labels_filepath,
)
(x_train, y_train), (x_test, y_test) = mnist_dataloader.load_data()